## All challenge text is excerpted from https://toadstyle.org/cryptopals/60.txt

```
60. Single-Coordinate Ladders and Insecure Twists

All our hard work is about to pay some dividends. Here's a list of
cool-kids jargon you'll be able to deploy after completing this
challenge:

* Montgomery curve
* single-coordinate ladder
* isomorphism
* birational equivalence
* quadratic twist
* trace of Frobenius

Not that you'll understand it all; you won't. But you'll at least be
able to silence crypto-dilettantes on Twitter.

Now, to the task at hand. In the last problem, we implemented ECDH
using a short Weierstrass curve form, like this:

    y^2 = x^3 + a*x + b

For a long time, this has been the most popular curve form. The NIST
P-curves standardized in the 90s look like this. It's what you'll see
first in most elliptic curve tutorials (including this one).

We can do a lot better. Meet the Montgomery curve:

    B*v^2 = u^3 + A*u^2 + u

Although it's almost as old as the Weierstrass form, it's been buried
in the literature until somewhat recently. The Montgomery curve has a
killer feature in the form of a simple and efficient algorithm to
compute scalar multiplication: the Montgomery ladder.

Here's the ladder:

    function ladder(u, k):
        u2, w2 := (1, 0)
        u3, w3 := (u, 1)
        for i in reverse(range(bitlen(p))):
            b := 1 & (k >> i)
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
            u3, w3 := ((u2*u3 - w2*w3)^2,
                       u * (u2*w3 - w2*u3)^2)
            u2, w2 := ((u2^2 - w2^2)^2,
                       4*u2*w2 * (u2^2 + A*u2*w2 + w2^2))
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
        return u2 * w2^(p-2)

You are not expected to understand this.

No, really! Most people don't understand it. Instead, they visit the
Explicit-Formulas Database (https://www.hyperelliptic.org/EFD/), the
one-stop shop for state-of-the-art ECC implementation techniques. It's
like cheat codes for elliptic curves. Worth visiting for the
bibliography alone.

With that said, we should try to demystify this a little bit. Here's
the CliffsNotes:

1. Points on a Montgomery curve are (u, v) pairs, but this function
   only takes u as an input. Given *just* the u coordinate of a point
   P, this function computes *just* the u coordinate of k*P. Since we
   only care about u, this is a single-coordinate ladder.

2. So what the heck is w? It's part of an alternate point
   representation. Instead of a coordinate u, we have a coordinate
   u/w. Think of it as a way to defer expensive division (read:
   inversion) operations until the very end.

3. cswap is a function that swaps its first two arguments (or not)
   depending on whether its third argument is one or zero. Choosy
   implementers choose arithmetic implementations of cswap, not
   branching ones.

4. The core of the inner loop is a differential addition followed by a
   doubling operation. Differential addition means we can add two
   points P and Q only if we already know P - Q. We'll take this
   difference to be the input u and maintain it as an invariant
   throughout the ladder. Indeed, our two initial points are:

       u2, w2 := (1, 0)
       u3, w3 := (u, 1)

   Representing the identity and the input u.

5. The return statement performs the modular inversion using a trick
   due to Fermat's Little Theorem:

       a^p     = a    mod p
       a^(p-1) = 1    mod p
       a^(p-2) = a^-1 mod p

6. A consequence of the Montgomery ladder is that we conflate (u, v)
   and (u, -v). But this encoding also conflates zero and
   infinity. Both are represented as zero. Note that the usual
   exceptional case where w = 0 is handled gracefully: our trick for
   doing the inversion with exponentiation outputs zero as expected.

   This is fine: we're still working in a subgroup of prime order.

Go ahead and implement the ladder. Remember that all computations are
in GF(233970423115425145524320034830162017933).

Oh yeah, the curve parameters. You might be thinking that since we're
switching to a new curve format, we also need to pick out a whole new
curve. But you'd be totally wrong! It turns out that some short
Weierstrass curves can be converted into Montgomery curves.

This is because all finite cyclic groups with an equal number of
elements share a kind of equivalence we call "isomorphism". It makes
sense, if you think about it - if the order is the same, all the same
subgroups will be present, and in the same proportions.

So all we need to do is:

1. Find a Montgomery curve with an equal order to our curve.

2. Figure out how to map points back and forth between curves.

You can perform this conversion algebraically. But it's kind of a
pain, so here you go:

    v^2 = u^3 + 534*u^2 + u

Through cunning and foresight, I have chosen this curve specifically
to have a really simple map between Weierstrass and Montgomery
forms. Here it is:

    u = x - 178
    v = y

Which makes our base point:

    (4, 85518893674295321206118380980485522083)

Or, you know. Just 4.

Anyway, implement the ladder. Verify ladder(4, n) = 0. Map some points
back and forth between your Weierstrass and Montgomery representations
and verify them.
```

In [1]:
from dataclasses import dataclass
from functools import reduce
from datetime import datetime
from itertools import count
from operator import mul
from random import Random, randrange
from math import log, ceil
from pprint import pprint

from challenge_31 import do_sha256, hmac
from from_notebook.challenge_57 import primegen, int_to_bytes, mini_crt, crt
from from_notebook.challenge_59 import modsqrt, NoQuadraticResidueError

In [2]:
A = 534
B = 1
P = 233970423115425145524320034830162017933
assert 2**127 < P < 2**128

In [3]:
# We'll define more Montgomery operations later, but let's start off with just the ladder

def ladder(u, k, a=A, b=B, p=P, blp=128):  # blp: bitlength of p
    u2, w2 = 1, 0
    u3, w3 = u, 1
    for i in range(blp-1, -1, -1):
        b = 1 & (k >> i)
        u2, u3 = (u3, u2) if b else (u2, u3)  # note: branching, not arithmetic, implementation of cswap
        w2, w3 = (w3, w2) if b else (w2, w3)  # used here for performance (runs appx 17% faster for some reason)
        u3, w3 = ((u2*u3 - w2*w3)**2 % p,
                   u * (u2*w3 - w2*u3)**2 % p)
        u2, w2 = ((u2**2 - w2**2)**2 % p,
                   4*u2*w2 * (u2**2 + a*u2*w2 + w2**2) % p)
        u2, u3 = (u3, u2) if b else (u2, u3)
        w2, w3 = (w3, w2) if b else (w2, w3)
    return (u2 * pow(w2, p-2, p)) % p

In [4]:
order = 233970423115425145498902418297807005944  # copied over from 59.txt
assert ladder(4, order) == 0
assert ladder(4, 0) == 0
print("Basic tests for Montgomery ladder passed.")

Basic tests for Montgomery ladder passed.


In [5]:
from from_notebook.challenge_59 import Curve as WeierstrassCurve, Point as WeierstrassPoint

w_curve = WeierstrassCurve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)
w_base = w_curve.point(182, 85518893674295321206118380980485522083)

def get_v_sq(u, p=P):
    return (pow(u, 3, p) + A*pow(u, 2, p) + u) % p

def w_to_m(pt):
    if pt is w_curve.zero:
        return 0
    return (pt.x - 178, pt.y)

def m_to_w(u):
    """For any nonzero u there are two possible values of v; this returns both of them."""
    if u == 0:
        return w_curve.zero
    
    v1, v2 = modsqrt(get_v_sq(u), P)
    return (w_curve.point(u + 178, v1),
            w_curve.point(u + 178, v2))

def point_conversion_tests():
    assert w_to_m(w_base)[0] == 4
    assert w_to_m(w_base*777)[0] == ladder(4, 777)

    assert w_base in m_to_w(4)
    assert w_base*1000 in m_to_w(ladder(4, 1000))

    # use a static rng seed for reproducibility
    r = Random(b"i brought all this so you can survive when law is lawless")
    for _ in range(50):
        i = r.randrange(1, order)
        j = r.randrange(1, order)
        w_pt = (w_base * i) * j
        m_pt = ladder(ladder(4, i), j)
        assert w_pt in m_to_w(m_pt)
        assert m_pt == w_to_m(w_pt)[0]

point_conversion_tests()
print("Further tests for Montgomery ladder passed.")

Further tests for Montgomery ladder passed.


```
One nice thing about the Montgomery ladder is its lack of special
cases. Specifically, no special handling of: P1 = O; P2 = O; P1 = P2;
or P1 = -P2. Contrast that with our Weierstrass addition function and
its battalion of ifs.

And there's a security benefit, too: by ignoring the v coordinate, we
take away a lot of leeway from the attacker. Recall that the ability
to choose arbitrary (x, y) pairs let them cherry-pick points from any
curve they can think of. The single-coordinate ladder robs the
attacker of that freedom.

But hang on a tick! Give this a whirl:

    ladder(76600469441198017145391791613091732004, 11)
```

In [6]:
ladder(76600469441198017145391791613091732004, 11)

0

```

Let's do a quick sanity check. Here's the curve equation again:

    v^2 = u^3 + 534*u^2 + u

Plug in u and take the square root to recover v.
```

In [7]:
try:
    modsqrt(get_v_sq(76600469441198017145391791613091732004), P)
except NoQuadraticResidueError:
    print("Square root does not exist!")

Square root does not exist!


```
You should detect that something is quite wrong. This u does not
represent a point on our curve! Not every u does.

This means that even though we can only submit one coordinate, we
still have a little bit of leeway to find invalid
points. Specifically, an input u such that u^3 + 534*u^2 + u is not a
quadratic residue can never represent a point on our curve. So where
the heck are we?

The other curve we're on is a sister curve called a "quadratic twist",
or simply "the twist". There is actually a whole family of quadratic
twists to our curve, but they're all isomorphic to each
other. Remember that that means they have the same number of points,
the same subgroups, etc. So it doesn't really matter which particular
twist we use; in fact, we don't even need to pick one.

We're mostly interested in the subgroups present on the twist, which
means we need to know how many points it contains. Fortunately, it
turns out to be easier to count the combined set of points on the
curve and its twist at the same time. Let's do it:

1. For every nonzero u up to the modulus p, if u^3 + A*u^2 + u is a
   square in GF(p), there are two points on the original curve.

2. If the above sum is a nonsquare in GF(p), there are two points on
   the twisted curve.

It should be clear that these add up to 2*(p-1) points in total, since
there are p-1 nonzero integers in GF(p) and two points for each. Let's
continue:

3. Both the original curve and its twist have a point (0, 0). This is
   just a regular point, not the group identity.

4. Both the original curve and its twist have an abstract point at
   infinity which serves as the group identity.

So we have 2*p + 2 points across both curves. Since we already know
how many points are on the original curve, we can easily calculate the
order of the twist.
```

In [8]:
twist_order = 2*P + 2 - order
twist_order

233970423115425145549737651362517029924

```
If Alice chose a curve with an insecure twist, i.e. one with a
partially smooth order, then some doors open back up for Eve. She can
choose low-order points on the twisted curve, send them to Alice, and
perform the invalid-curve attack as before.

The only caveat is that she won't be able to recover the full secret
using off-curve points, only a fraction of it. But we know how to
handle that.

So:

1. Calculate the order of the twist and find its small factors. This
   one should have a bunch under 2^24.
```

In [9]:
print("Factoring...")
%time factors = [p for p in primegen(up_to=2**24) if twist_order % p == 0 and (twist_order // p) % p != 0]
print("Small, non-repeated factors:", factors)

Factoring...
CPU times: user 17.5 s, sys: 282 ms, total: 17.8 s
Wall time: 17.9 s
Small, non-repeated factors: [11, 107, 197, 1621, 105143, 405373, 2323367]


```
2. Find points with those orders. This is simple:

   a. Choose a random u mod p and verify that u^3 + A*u^2 + u is a
      nonsquare in GF(p).

   b. Call the order of the twist n. To find an element of order q,
      calculate ladder(u, n/q).
```

In [10]:
def get_twist_point(factor):
    while True:
        u = randrange(0, P)
        expr = (pow(u, 3, P) + A*pow(u, 2, P) + u) % P
        try:
            modsqrt(expr, P)  # this try clause checks to make sure u isn't on the curve
        except NoQuadraticResidueError: pass
        else: continue
        elem = ladder(u, twist_order // factor)
        if elem != 0:
            break

    assert twist_order % factor == 0
    assert ladder(elem, factor) == 0

    return elem

twist_points = {fac: get_twist_point(fac) for fac in factors}
pprint(twist_points)

{11: 105888069003703096891937904030103459645,
 107: 88496758108129352636311393052988812973,
 197: 95847374294096097240229113944596795912,
 1621: 209040869111920169375451605921851765524,
 105143: 16691963560151280031096076764548932715,
 405373: 52072102360943813551160493462411631079,
 2323367: 143719522380709265725703217313534480142}


```
3. Send these points to Alice to recover portions of her secret.
```

In [11]:
# We'll implement alice as a coroutine, just like Bob from challenges 58 & 59

def get_tag(message, secret: int):
    mac_key = do_sha256(int_to_bytes(secret))
    mac_tag = hmac(mac_key, message)
    return mac_tag

def alice_coro(message):
    priv = randrange(0, order)
    pub = ladder(4, priv)
    
    print("Alice: Private key =", priv)

    h = (yield pub)
    while True:
        secret = ladder(h, priv)
        t = get_tag(message, secret)
        h = (yield (message, t))

alice = alice_coro(b"no alarms and no surprises")
alice_pubkey = next(alice)
print("Alice initialized. Pubkey:", alice_pubkey)

Alice: Private key = 39266882933223832617786989698524938986
Alice initialized. Pubkey: 175508529584355732225164217236728349325


In [12]:
def recover_residue(base, order, message, tag):
    # Recovers the residue of Alice's secret mod `order`
    for i in range(order):
        if get_tag(message, ladder(base, i)) == tag:
            return (i, order-i)  # two possibilities
    raise Exception("residue not found (?!)")  # this should not ever happen

In [13]:
residues = {}

print("This may take several minutes.")
for small_order, small_pt in twist_points.items():
    print("\nRecovering possible residues mod", small_order, "...", flush=True)
    message, t = alice.send(small_pt)
    %time i1, i2 = recover_residue(small_pt, small_order, message, t)
    residues[small_order] = i1, i2
    print(i1, i2)

print("Done!\n")
pprint(residues)

This may take several minutes.

Recovering possible residues mod 11 ...
CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 2.58 ms
5 6

Recovering possible residues mod 107 ...
CPU times: user 17.6 ms, sys: 12 µs, total: 17.6 ms
Wall time: 17.9 ms
40 67

Recovering possible residues mod 197 ...
CPU times: user 9.59 ms, sys: 0 ns, total: 9.59 ms
Wall time: 9.83 ms
21 176

Recovering possible residues mod 1621 ...
CPU times: user 427 ms, sys: 2.1 ms, total: 429 ms
Wall time: 434 ms
386 1235

Recovering possible residues mod 105143 ...
CPU times: user 21 s, sys: 390 µs, total: 21 s
Wall time: 21.1 s
46210 58933

Recovering possible residues mod 405373 ...
CPU times: user 1min 48s, sys: 64.1 ms, total: 1min 48s
Wall time: 1min 50s
77077 328296

Recovering possible residues mod 2323367 ...
CPU times: user 11min 52s, sys: 0 ns, total: 11min 52s
Wall time: 12min
1043913 1279454
Done!

{11: (5, 6),
 107: (40, 67),
 197: (21, 176),
 1621: (386, 1235),
 105143: (46210, 58933),
 405373

```
4. When you've exhausted all the small subgroups in the twist, recover
   the remainder of Alice's secret with the kangaroo attack.

HINT: You may come to notice that k*u = -k*u, resulting in a
combinatorial explosion of potential CRT outputs. Try sending extra
queries to narrow the range of possibilities.
```

In [14]:
# Just for kicks, here's the order-11 twist subgroup we found:

for i in range(12):
    print(ladder(twist_points[11], i))
print("...")

# note the symmetry: the subgroup's i'th and 11-i'th elements are all equal
# this is just a different way of stating what was alluded to in the hint above

0
105888069003703096891937904030103459645
1430388126279164727092494211327512206
76600469441198017145391791613091732004
4612483201341222105440076661179035958
173527332646559565669040569905840307859
173527332646559565669040569905840307859
4612483201341222105440076661179035958
76600469441198017145391791613091732004
1430388126279164727092494211327512206
105888069003703096891937904030103459645
0
...


In [15]:
# We've got two possible residues mod 11, two more mod 107, etc.
# We can use the Chinese Remainder Theorem to combine these possibilities together.
# With a naive approach, this gives us 2**7 possible combinations of residues.
# However, we can prevent combinatorial explosion by testing and disqualifying candidate
# residues as we go, like so:

In [16]:
def crt_step(m1, r11, r12, m2, r21, r22):
    m = m1*m2  # new combined modulus
    candidates = [crt((r11, r21), (m1, m2))[0],
                  crt((r11, r22), (m1, m2))[0],
                  crt((r12, r21), (m1, m2))[0],
                  crt((r12, r22), (m1, m2))[0]]

    while True:
        pt = get_twist_point(m)
        if ladder(pt, m1) != 0 and ladder(pt, m2) != 0:
            break

    msg, tag = alice.send(pt)
    r1, r2 = [r for r in candidates if get_tag(msg, ladder(pt, r)) == tag]
    return m, (r1, r2)

modulus, candidates = reduce(lambda a, b: crt_step(a[0], *a[1], b[0], *b[1]),
                             residues.items())
print("Alice's private key is congruent to:")
print(candidates[0], "or", candidates[1])
print("mod", modulus)

Alice's private key is congruent to:
19525304426922560702000460 or 17694895688627123677402577
mod 37220200115549684379403037


In [17]:
# To run the Kangaroo attack we need more than just a ladder - we need a full
# Montgomery curve implementation. So let's write that now.

# reference for formulas: https://www.hyperelliptic.org/EFD/g1p/auto-montgom.html

In [18]:
@dataclass
class MontgomeryCurve:
    a: int
    b: int
    p: int
    
    zero = object()
    
    def ladder(self, u, k):
        return ladder(u, k, self.a, self.b, self.p)
    
    def inv(self, uv):
        u, v = uv
        p = self.p
        return (u, p-v)  # = (u, -v) mod p
    
    def add(self, uv1, uv2):
        # special cases: either point is zero
        if uv1 == (0, 1): return uv2
        if uv2 == (0, 1): return uv1

        u1, v1 = uv1
        u2, v2 = uv2

        # special cases: points are equal or are inverses
        if u1 == u2:
            if v1 == v2:
                return self.double(uv1)
            else:
                assert v2 == -v1 % self.p
                return (0, 1)

        # generic case
        a, b, p = self.a, self.b, self.p
        
        du = (u2-u1) % p
        dv = (v2-v1) % p
        dui = pow(du, -1, p)

        u3 = (b * pow(dv, 2, p) * pow(dui, 2, p) - a - u1 - u2) % p
        v3 = ((2*u1 + u2 + a)*dv*dui - b*pow(dv, 3, p)*pow(dui, 3, p) - v1) % p

        return (u3, v3)
    
    def sub(self, uv1, uv2):
        return self.add(uv1, self.inv(uv2))
    
    def get_v_from_u(self, u):  # returns both possible values for v
        a, b, p = self.a, self.b, self.p
        assert b == 1  # simplifies the following eqns
        rhs = (pow(u, 3, p) + a*pow(u, 2, p) + u) % p
        v_sq = (rhs * pow(b, -1, p)) % p
        v1, v2 = modsqrt(rhs, p)
        return v1, v2
    
    def mul(self, pt, k):
        # computes ladder(u, k) to get (pt*k).u, then computes both candidate
        # values for v. To figure out which v-coordinate is correct, we take one,
        # add pt to it, and check the result against ladder(pt, k+1). The total
        # cost is two ladders, one point addition, and one modsqrt. This is sort
        # of hacky and inefficient, but it works, and it's much simpler than the
        # full two-coordinate ladder.
        u = pt[0]
        uk = self.ladder(u, k)
        ukpp = self.ladder(u, k+1)
        v1, v2 = self.get_v_from_u(uk)
        if self.add(pt, (uk, v1))[0] == ukpp:
            return (uk, v1)
        else:
            return (uk, v2)
    
    def point(self, u, v):
        return Point(u, v, self)
    
    def double(self, uv):
        if uv == (0, 1): return uv

        a, b, p = self.a, self.b, self.p
        u, v = uv
        usq = pow(u, 2, p)
        
        u3 = (b*pow(3*usq + 2*a*u+1, 2, p) * pow(2*b*v, -2, p)-a-u-u) % p
        v3 = ((2*u+u+a)*(3*usq+2*a*u+1)*pow(2*b*v, -1, p) - b*pow(3*usq+2*a*u+1, 3, p)*pow(2*b*v, -3, p) - v) % p
        
        return (u3, v3)


@dataclass
class Point:
    u: int
    v: int
    curve: MontgomeryCurve

    def coords(self):
        return self.u, self.v

    def __neg__(self):
        curve = self.curve
        u_inv, v_inv = curve.inv(self.coords())
        return curve.point(u_inv, v_inv)

    def __add__(self, other):
        curve = self.curve
        zero = curve.zero
        assert isinstance(other, Point) or other is zero
        p1 = self.coords()
        p2 = other.coords()
        result = curve.add(p1, p2)
        return result if result is zero else curve.point(*result)

    def __sub__(self, other):
        if other is self.curve.zero:
            return self
        assert isinstance(other, Point)
        return self + (-other)

    def __mul__(self, k):
        assert isinstance(k, int)
        curve = self.curve
        uv = self.coords()
        result = curve.mul(uv, k)
        return curve.point(*result)
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other):
        return self - other
    
    def __rmul__(self, other):
        return self * other

In [19]:
CURVE = MontgomeryCurve(A, B, P)
G = CURVE.point(4, 85518893674295321206118380980485522083)

In [20]:
assert 10*G == G*10
assert (12345*G).u == ladder(4, 12345)

In [21]:
# now, here's how that funky transform from 58.txt maps to the elliptic curve setting

# alice has a private key x and a public key y = x*G
# we know y, and we have two possible values for x's residue mod some big modulus r
# we'll consider each value in turn, denoting it as n, i.e. x = n mod r

# x = n mod r => n + m*r for some m
# y = x*G => y = (n + m*r)*G
# y' := y - n*G = (m*r)*G
# G' := r*G
# y' = m*G'

# knowing y' and G', pollard's algorithm uses the fact that y' = m*G' to find m
# then once we find m, we can use x = n + m*r to find x

def transform(n, r, y, G):
    """
    Given n, r, y, G, returns (G', y')
    """
    return r*G, y-(n*G)

In [28]:
def ec_pollard(G, y, a, b, k=24):
    """
    G: base point
    y: point whose index we want to find
    a: lower bound of search range
    b: upper bound of search range
    k: jump size limit (default value should be fine)
    
    Returns m such that y = m*G
    """
    
    N = 4 * (2**k - 1) // k
    pt_cache = [G * (2**u) for u in range(k)]  # cache these to avoid computing them inside the hot loop
    update_interval = 2**21 - 1  # interval between human-friendly status updates
    
    print(f"\nRunning Pollard's algorithm ({k=}, {N=})")
    print("Tame kangaroo starting at:", datetime.now())
    
    # this is the tame kangaroo loop
    xT = 0
    yT = b * G
    for i in range(N):
        # inline f() for performance
        f_yT = 1 << (u := yT.u % k)
        xT += f_yT
        yT += pt_cache[u]
    yT_u = yT.u  # bring this value into the local namespace to avoid costly lookups
    
    print("Wild kangaroo starting at:", t0 := datetime.now())
    
    # this is the wild kangaroo loop
    xW = 0
    yW = y
    bound = b - a + xT
    for i in count():
        if xW > bound:
            break
        
        if i & update_interval == 0:
            progress = xW / bound
            if (progress > 0):
                ti = datetime.now() - t0
                est_duration = ti / progress
                print(f"{datetime.now()} | {100*progress:f}% done | Appx time remaining: ({est_duration - ti} / {est_duration})")

        f_yW = 1 << (u := yW.u % k)
        xW += f_yW
        yW += pt_cache[u]

        if yW.u == yT_u:
            print("The wild kangaroo found something!")
            return b + xT - xW

In [29]:
def get_point_index(residue, modulus, y, G, order, k=24):
    curve = G.curve
    lb, ub = 0, (order // modulus) // 2  # search bounds (note we only need to search
                                         # the first half of the range due to symmetry)
    
    G_prime, y_prime = transform(residue, modulus, y, G)
    m = ec_pollard(G_prime, y_prime, lb, ub, k)
    if m is not None:
        return residue + m*modulus

In [30]:
n1, n2 = candidates  # candidate residues
y1, y2 = [CURVE.point(alice_pubkey, v) for v in CURVE.get_v_from_u(alice_pubkey)]

# this loop increments k so that we can retry the kangaroo algorithm with different params
# if it fails for both residues (which might happen, since it's a probabilistic algorithm)
for k in count(24):
    x = (get_point_index(n1, modulus, y1, G, order, k) or
         get_point_index(n2, modulus, y1, G, order, k) or
         get_point_index(n1, modulus, y2, G, order, k) or
         get_point_index(n2, modulus, y2, G, order, k))
    if x is not None:
        assert (x*G).u == alice_pubkey
        print("Alice's private key is", x)
        break
    print("No luck this time; incrementing k and trying again.\n")


Running Pollard's algorithm (k=24, N=2796202)
Tame kangaroo starting at: 2021-05-01 21:23:14.112487
Wild kangaroo starting at: 2021-05-01 21:24:01.011696
2021-05-01 21:24:44.331960 | 28.787346% done | Appx time remaining: (0:01:47.163394 / 0:02:30.483640)
2021-05-01 21:25:28.472596 | 57.539115% done | Appx time remaining: (0:01:04.541600 / 0:02:32.002485)
The wild kangaroo found something!
Alice's private key is 39266882933223832617786989698524938986
